# Chatbot y Text-To-Speech

* Bibliotecas necesarias para la ejecución del código

In [11]:
# Python 3.7
#!pip install chatterbot
#!pip install chatterbot_corpus
#!pip install gtts
#!python3 -m spacy download es
#!pip install spacy==2.3.5
#!python -m spacy download es_core_news_lg

## La clase principal

In [13]:
import os
# Chatterbot
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer, ChatterBotCorpusTrainer
import es_core_news_lg
nlp = es_core_news_lg.load()
# TTS
from gtts import gTTS
from gtts.tokenizer import pre_processors, tokenizer_cases
import gtts.tokenizer.symbols

class My_bot:
    def __init__(self, name='IA'):
        # Nuestro chatBot
        self.chatbot = ChatBot(name, logic_adapters=['chatterbot.logic.MathematicalEvaluation', 'chatterbot.logic.BestMatch'])
        self.chatbot_forget()
        # Text_to_speech
        #self.tts_train_symbols()
        self.tok = self.tts_token()
        
    def chatbot_basic_train(self):
        # Entrenamiento básico (compus)
        trainer = ChatterBotCorpusTrainer(self.chatbot)
        trainer.train(
            "chatterbot.corpus.spanish.greetings",
            "chatterbot.corpus.spanish.conversations"
        )
        
    def chatbot_spec_train(self, filename='./specific_train/custom.yml'):
        # Entrenamiento específico
        trainer = ChatterBotCorpusTrainer(self.chatbot)
        trainer.train(filename)
        
    def chatbot_forget(self):
        self.chatbot.storage.drop()
        
    def chatbot_response(self, query):
        response = self.chatbot.get_response(query)
        return response
    
    # ---------- Específico para comentar código (future work) ----
    def tts_train_symbols(self):
        cod_text = [(",", "coma,"), 
                    (".", "punto,"), 
                    (";", "punto y coma,"), 
                    (":", "dos puntos,"), 
                    ("(", "abrimos paréntesis,"),
                    (")", "cerramos paréntesis,"),
                    ("[", "abrimos corchertes,"),
                    ("]", "cerramos corchetes,")]
        for item in cod_text:
            gtts.tokenizer.symbols.SUB_PAIRS.append(item)
    # -------------------------------------------------------------
            
    def tts_token(self):
        return gtts.tokenizer.core.Tokenizer([
            tokenizer_cases.tone_marks,
            tokenizer_cases.period_comma,
            tokenizer_cases.other_punctuation])
    
    def tts_say_this(self, string, filename='say.mp3'):
        string = pre_processors.word_sub(string)
        gtts = gTTS(text=string, lang='es')
        gtts.save(filename) 
        
    def chatbot_response_to_mp3(self, query, filename='response.mp3'):
        response = self.chatbot_response(query)
        print(response)
        #query = pre_processors.word_sub(response.text)
        gtts = gTTS(text=response.text, lang='es')
        gtts.save(filename) 
        

# Ejemplo de uso
# bot = my_bot()
# bot.chatbot_basic_train()
# bot.chatbot_response_to_mp3('Hola, ¿qué tal?', 'test.mp3')

### Ejemplo ejercicio 1 - Code Soldier
* La idea principal
**Narrador:** Pregunta + posibles respuestas
**chatbot:** Ayuda personalizada, se le puede pedir pistas, enlaces, etc

In [5]:
# Iniciamos la clase my_bot
bot = My_bot()
bot.chatbot_forget()
bot.chatbot_basic_train()

Training greetings.yml: [####################] 100%


In [14]:
# Entrenamiento específico para el ejercicio 1 - (pistas, comentarios posibles, etc.)
bot.chatbot_spec_train(filename='./specific_train/ejercicio_1_Code_Soldier.yml')

Training ejercicio_1_Code_Soldier.yml: [####################] 100%


El narrador se encarga de describir la pregunta (mediante TextToSpeech, guarda los archivos en formato mp3 por compatibildad en Front-End)

In [15]:
# Ejercicio 1 - ejercicio_1_Code_Soldier.yml
bot.tts_say_this("¿Qué tipos de datos básicos existen en python?", "./audio/query.mp3")

In [18]:
# Las posibles respuestas
bot.tts_say_this("Opción A: Números, caracteres y buleanos", "./audio/solA.mp3")
bot.tts_say_this("Opción B: Enteros y Cadenas de texto", "./audio/solB.mp3")
bot.tts_say_this("Opción C: Números y cadenas de texto", "./audio/solC.mp3")

Probamos el ChatterBot (en conjunto con TTS)

In [19]:
text = "Dame una pista"
bot.chatbot_response_to_mp3(text, filename='./audio/response.mp3')

In [20]:
text = "¿Que tipo de datos númericos existen en python?"
bot.chatbot_response_to_mp3(text, filename='./audio/response.mp3')

## Future Works

### SpeechRecognition
Reconocimiento de voz para la total implementación y facilitación de acceso a personas con capacidades disminuidas

In [21]:
# !pip install SpeechRecognition
# !pip3 install pyaudio
# !python3 -m pip install sounddevice && python3 -m sounddevice

(Proyecto no acabado)

In [ ]:
import speech_recognition as sr
import pyaudio

In [ ]:
def listen():
    
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Ahora puedes hablar")
        r.adjust_for_ambient_noise(source)
        audio = r.listen(source)
        
    print("Translating your speech...")
    r.recognize_google(audio, language='es-ES')
    print("You said: " + r.recognize_google(audio))

### Implantación de SST para código
Proyecto de futuro para la implementación de código guiado por voz, con el uso de enfasis oral en los distintos signos de puntuación o tabulación (importante para el mundo de la programación)

In [10]:
#
#    cod_text = [(",", "coma,"), 
#                (".", "punto,"), 
#                (";", "punto y coma,"), 
#                (":", "dos puntos,"), 
#                ("(", "abrimos paréntesis,"),
#                (")", "cerramos paréntesis,"),
#                ("[", "abrimos corchertes,"),
#                ("]", "cerramos corchetes,")]
#    for item in cod_text:
#        gtts.tokenizer.symbols.SUB_PAIRS.append(item)
#